In [8]:
import json
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import pandas as pd

In [2]:
df = pd.read_csv('cleaned.csv')

In [3]:
dat = pd.read_csv('date.csv')

In [4]:
dat['Date'] = pd.to_datetime(dat['Date'])

In [5]:
dat = dat.sort_values(by='Date')

In [6]:
dat['Cummulative'] = dat['Occurance'].cumsum()

In [18]:
px.scatter?

In [10]:
import plotly.graph_objects as go

In [11]:
colors = {
    'background': '#98AFC7',
    'text': '#111111'}
barcolor = '#f0bf60'


ranges = [1, 30, 50, 100, 200, 3800 ]
# Area against count
area_count = px.histogram(df, x = 'AREA', hover_data =['AREA'], title='Number of Motors per Area', color_discrete_sequence=[barcolor])
area_count.update_layout(clickmode='event+select')
area_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)


# RATINGS RANGE
ran = df['RATING'].value_counts(bins=ranges, sort=False).reset_index()
ran = ran.rename(columns={"index": "Ratings Range", "RATING": "Number"})
pd.DataFrame(ran)
rang = ['1-30', '30-50', '50-100', '100-200', '200-3800']
ran['Range'] = rang
rat_count = px.bar(ran, x='Range', y='Number', title='Number of Motors per Range', color_discrete_sequence=[barcolor])
rat_count.update_layout(clickmode='event+select')
rat_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)


# STARTERS AGAINST RATING
star_rat = px.histogram(df, x='STARTER', title='Number of Motors', color_discrete_sequence=[barcolor])
star_rat.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# Trend of power outages
outage = go.Figure(data=go.Scatter(x=dat['Date'], y=dat['Cummulative'], mode='lines+markers'))
outage.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app = JupyterDash(__name__)

locations = df['AREA'].unique()

app.layout = html.Div([
   
     dcc.Graph(id="area_c", figure = area_count,
            style= {'width': '49%', 'display': 'inline-block',}),
    
    dcc.Graph(id="area_r", figure = rat_count,
            style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="start_c", figure = star_rat,
            style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="trend", figure = outage,
            style= {'width': '49%', 'display': 'inline-block'})    

])
    
@app.callback(
     Output('start_c', 'figure'),
    [Input('area_c', 'clickData'),
     Input('area_r', 'clickData')])

def display_click_data(counts, ratings):

    if (counts == None) & (ratings == None):
        
        
        return star_rat
    
    elif (counts == None) & (ratings != None):
        rats = ratings['points'][0]['x'].split('-')[0]
        dff= df.copy()
        dff = dff.loc[(dff['RATING'] > int(rats)) ]
        # NUMBER OF STARTERS
        star_ratt = px.histogram(dff, x='STARTER', title='Graph of Number of Motor Starters', color_discrete_sequence=[barcolor])
        star_ratt.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
)
        return star_ratt
    
    elif (counts != None) & (ratings == None):
        coun = counts['points'][0]['x']
        dff= df.copy()
        dff = dff.loc[(dff['AREA'] == coun) ]
        star_ratt = px.histogram(dff, x='STARTER', title='Graph of Number of Motor Starters', color_discrete_sequence=[barcolor])
        star_ratt.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
)
        return star_ratt
    
    else:
        rats = ratings['points'][0]['x'].split('-')[0]
        coun = counts['points'][0]['x']
        dff= df.copy()
        dff = dff.loc[(dff['AREA'] == coun) & (dff['RATING'] > int(rats)) ]
        # NUMBER OF STARTERS
        star_ratt = px.histogram(dff, x='STARTER', title='Graph of Number of Motor Starters', color_discrete_sequence=[barcolor])
        star_ratt.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
)

        
        return star_ratt



if __name__ == '__main__':
    app.run_server(debug=True,  port=8051)



Dash app running on http://127.0.0.1:8051/


In [139]:
df.loc[(df['AREA']== 'BIOX') & (df['RATING'] >100) & (df['STARTER'] == 'DOL')]

,Unnamed: 0,NUMBER,NAME,RATING,STARTER,FUNCTION,AREA
2,119,23-AG-143,REACTOR TANK 143 AGITATOR,225.0,DOL,AGITATOR,BIOX
10,126,23-AG-145,REACTOR TANG 145 AGITATOR,132.0,DOL,AGITATOR,BIOX
13,130,23-AG-146,REACTOR TANK 146 AGITATOR,132.0,DOL,AGITATOR,BIOX
15,132,23-AG-147,REACTOR TANK 147 AGITATOR,132.0,DOL,AGITATOR,BIOX
20,113,23-AG-036,REACTOR TANK AGITATOR,132.0,DOL,AGITATOR,BIOX
21,112,23-AG-034,REACTOR TANK AGITATOR,225.0,DOL,AGITATOR,BIOX
22,111,23-PP-035,REACTOR TANK 55 AGITATOR,225.0,DOL,PUMP,BIOX
36,104,23-AG-031,REACTOR TANK 31 AGITATOR,225.0,DOL,AGITATOR,BIOX
37,105,23-AG-032,REACTOR TANK 32 AGITATOR,225.0,DOL,AGITATOR,BIOX
38,106,23-AG-033,REACTOR TANK 33 AGITATOR,225.0,DOL,AGITATOR,BIOX
